# This is the code which solves the Static Traffic Assignment Problem

We want to solve the following problem (TAP-C):
\begin{align}
\min_{f, h} &\sum_{a} \int_{0}^{f_a} c_a(s)\; \text{d}s
\\
\text{s.t.  } & \;\; f = \Delta h
\\
& \;\; 
h \geq 0
\\
& \;\; 
A h = d
\\
& \;\; 
f \leq u
\end{align}

This problem is hard to compute, even if it is a convex problem. Computing all the path of a network is NP-hard (with respect to the number of edges and links).

Nevertheless, the TAP can be solve using a Frank-Wolf algorithm:
\begin{align}
&\text{1. TO DO }
\\
&\text{2. }
\end{align}


#### Remarks
First, we tried to solve the STA with CVX.
This does not work because the values of the travel time are to big. 

Then, we solves the STA using links flow. 
This does not help us in our project because we need the path flow to compute the dual of the TAP-C.

Finally, we solve the STA using paths flow.
The issue here is that finding every path in a network in a NP-complete problem.
But if we can compute every path (we only need to do it one time), then it is really fast to compute the shortest path. We do not need to do a Dikjstra's algorithm to compute the shortest path, only a big matrix multiplication and a array sorting are enough. This might be more difficult on a laptop. But it might be faster on a HPC.

An other possibility, would be to compute the paths during the Frank Wolf's algorithm. THIS HAS TO BE DONE.

### I. We load and clean the network and the demand data

In [1]:
import numpy as np
import scipy
import scipy.sparse

In [113]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network = I210
debug = True

In [114]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [115]:
"""
We need to demand to be an array like:
[
[o1, d1, demand from o1 to d1],
...,
[on, dn, demand from on to dn]
]

We need the graph to be an array like:
[
[id link 1, node origin link 1, node destination link 1, a1, a2, a3, a4],
...,
[id link n, node origin link n, node destination link n, a1, a2, a3, a4]
]
where the node are indexed from 0 to nb_nodes-1,
    the links are indexed from 0 to nb_links-1

One can add some checks here to be sure that demand and graph respect this format!
"""

def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix (2d array)
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    return graph, demand

graph, demand = cleaning_input(graph, demand)
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
nb_ods = int(demand.shape[0])
if debug:
    print("nb nodes = " + str(nb_nodes))
    print("nb links = " + str(nb_links))
    print("nb ods = " + str(nb_ods))

nb nodes = 20
nb links = 41
nb ods = 1


### 2. We compute the incidence matrix

In [116]:
"""
We need to define 2 matrix called path_matrix (or delta) and route2od which gives are:
- delta is the incidence matrix. Each row of delta is a path, each column a link.
    delta[i][j] is 1 if the link j is used by the path i, 0 otherwise
- route2od is the array which recover the origin and destination of every path
    route2od[i] is the indice of the line which codes the origin and the demand
"""

from collections import defaultdict 
   
# This class represents a directed graph  
# using adjacency list representation 
class Graph: 
   
    def __init__(self,vertices): 
        #No. of vertices 
        self.V= vertices  
        # default dictionary to store graph 
        self.graph = defaultdict(list)  
   
    # function to add an edge to graph 
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
   
    def printAllPathsUtil(self, u, d, visited, path, graph_dict): 
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
        
        if u == d: 
            path_tmp = np.zeros(shape=(self.V))
            for i in range(len(path)-1):
                # link_indice = list(G.todok().keys()).index((path[i], path[i+1]))
                link_indice = graph_dict[path[i]][path[i+1]]
                path_tmp[link_indice] = 1
            if self.paths_m.shape[0]==0:
                self.paths_m = path_tmp.reshape((1, self.V))
            else:
                self.paths_m = np.append(self.paths_m, path_tmp.reshape((1, self.V)), axis=0)
        else: 
            # If current vertex is not destination 
            # Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]==False: 
                    self.printAllPathsUtil(i, d, visited, path, graph_dict) 
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop()
        visited[u]= False
   
   
    # Prints all paths from 's' to 'd' 
    def printAllPaths(self,s, d, graph_dict): 
        self.paths_m = np.array([])
        # Mark all the vertices as not visited 
        visited =[False]*(self.V)
  
        # Create an array to store paths 
        path = [] 
        
        # Call the recursive helper function to print all paths 
        self.printAllPathsUtil(s, d, visited, path, graph_dict) 
        return self.paths_m 

def return_graph_dict(graph):
    # graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
    graph_dict = {}
    for i in range(graph.shape[0]):
        try: 
            graph_dict[int(graph[i][1])]
        except:
            graph_dict[int(graph[i][1])] = {}
        graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    return graph_dict
    
# argument graph
def delta_matrix(graph, demand):
    route2od = np.array([])
    paths_matrix = np.array([])
    graph_dict = return_graph_dict(graph)
    g = Graph(graph.shape[0]) 
    for line in graph:
        g.addEdge(int(line[1]), int(line[2]))

    for i in range(demand.shape[0]):
        s = int(demand[i][0]) ; d = int(demand[i][1])
        paths_matrix_tmp = g.printAllPaths(s, d, graph_dict)
        route2od_tmp = np.ones(paths_matrix_tmp.shape[0])
        route2od_tmp = route2od_tmp*i
        if paths_matrix.shape[0]==0:
            paths_matrix = paths_matrix_tmp
        else:
            paths_matrix = np.append(paths_matrix, paths_matrix_tmp, axis=0)
        route2od = np.append(route2od, route2od_tmp)
    return paths_matrix, route2od

### 3. We define the travel time function. We encode the capacity here.

In [117]:
import sys
max_float = 1e+10
if debug:
    print(max_float)

# TO DO ADD THE CAPACITY
def travel_time(f, c = -1):
    if c == -1:
        c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    # here we need to have the same indexation for graph and f.
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else max_float for i in range(len(f))]
    return tt_tmp

10000000000.0


In [118]:
def update_travel_time(tt, G):
    for i in range(graph.shape[0]):
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]
    return G

In [119]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
G = update_travel_time(travel_time(np.zeros(nb_links)), G) # , [1,1,-1,1,1]))
# the following line make the matrix sparse, one can use G.toarray() to do the opposite
G = scipy.sparse.csr_matrix(G)

In [120]:
delta, route2od = delta_matrix(graph, demand)
if debug:
    print(delta.shape)
    print(delta[0])

(243, 41)
[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]


In [121]:
tt = travel_time(np.zeros(nb_links))
delta_csr = scipy.sparse.csr_matrix(delta)
if debug:
    print((delta_csr @ tt))
    print(np.argmin(delta_csr @ tt))

[10.7   9.17 11.59 11.96  8.35 10.77 13.76 10.15 11.57 11.78 10.25 12.67
 10.94  7.33  9.75 12.74  9.13 10.55 13.74 12.21 14.63 12.9   9.29 11.71
 13.74 10.13 11.55 12.07 10.54 12.96 13.33  9.72 12.14 15.13 11.52 12.94
 10.89  9.36 11.78 10.05  6.44  8.86 11.85  8.24  9.66 12.85 11.32 13.74
 12.01  8.4  10.82 12.85  9.24 10.66 13.74 12.21 14.63 15.   11.39 13.81
 16.8  13.19 14.61 12.56 11.03 13.45 11.72  8.11 10.53 13.52  9.91 11.33
 13.76 12.23 14.65 12.92  9.31 11.73 13.76 10.15 11.57  9.79  8.26 10.68
 11.05  7.44  9.86 12.85  9.24 10.66 10.87  9.34 11.76 10.03  6.42  8.84
 11.83  8.22  9.64 12.83 11.3  13.72 11.99  8.38 10.8  12.83  9.22 10.64
  8.88  7.35  9.77 10.14  6.53  8.95 11.94  8.33  9.75  7.7   6.17  8.59
  6.86  3.25  5.67  8.66  5.05  6.47  9.66  8.13 10.55  8.82  5.21  7.63
  9.66  6.05  7.47 10.55  9.02 11.44 11.81  8.2  10.62 13.61 10.   11.42
  9.37  7.84 10.26  8.53  4.92  7.34 10.33  6.72  8.14 10.57  9.04 11.46
  9.73  6.12  8.54 10.57  6.96  8.38 11.44  9.91 12

### 3. We upload the flow allocation 
#### 3.1 We compute the travel time of each path. 
#### 3.2 We compute the all or nothing allocation by putting all the demand on shortest paths.
#### 3.3 We update the flow allocation with a bisection.

In [52]:
"""
The function potential is used to compute the line search between 
the all or nothing allocation and the current flow allocation
The function potential returns the objective function corresponding to
the flow allocation f.
"""

def potential(graph, f, c=-1):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    if c == -1:
        c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    # here we need to have the same indexation for graph and f.
    pot_tmp = graph[:,3]*f + 1/2*graph[:,4]*(f**2) + 1/3*graph[:,5]*(f**3) + 1/4*graph[:,6]*(f**4) + 1/5*graph[:,7]*(f**5)
    pot_tmp = [pot_tmp[i] if f[i]<c[i] else f[i]*max_float for i in range(len(f))]
    return np.sum(pot_tmp)

    # return np.sum(graph[:,3]*f + 1/2*graph[:,4]*(f**2) + 1/3*graph[:,5]*(f**3) + 1/4*graph[:,6]*(f**4) + 1/5*graph[:,7]*(f**5))
    #links = int(np.max(graph[:, 0]) + 1)
    #g = np.copy(
    #    graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    #x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    #return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if debug:
            print(potential(graph, (1. - a) * f_tmp.toarray()[0] + a * f_aon.toarray()[0], c))
            print(l * d, end=": ")
            print(f(l * d))
            print(r * d, end=": ")
            print(f(r * d))
            print(str(m1 * d) + " = " + str(f(m1 * d)))
            print(str(m2 * d) + " = " + str(f(m2 * d)))
            print()
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

In [76]:
# check sparse vector
h = scipy.sparse.lil_matrix(np.zeros(delta_csr.shape[0]))
# the following is for one od
demand[0][2] = 10
# TO DO MODIFY TO TAKE INTO ACCOUNT SEVERALS OD
h[:,0] = demand[0][2]
print(h)

  (0, 0)	10.0


In [82]:
def compute_initial_flow(demand):
    return h
def solver(eps, nb_iter, demand, delta_csr, c=-1):
    h = compute_initial_flow(demand)
    for i in range(nb_iter):
        f = h @ delta_csr
        tt_flow = travel_time(f.toarray()[0], c)
        tt_p = delta_csr @ tt_flow
        # the following is for one od
        p_aon = np.argmin(tt_p)
        # the following is for one od
        # TO MODIFY TO TAKE INTO ACCOUNT SEVERAL ODs
        h_aon = scipy.sparse.lil_matrix(np.zeros(delta_csr.shape[0]))
        h_aon[:, p_aon] = demand[0][2]
        f_aon = h_aon @ delta_csr
        t = potential(graph, f.toarray()[0])
        s = line_search(lambda a: potential(graph, (1. - a) * f.toarray()[0] + a * f_aon.toarray()[0], c))
        h = (1.-s)*h + s*h_aon
        if s < eps:
            break
        if i % (nb_iter/20) == 0:
            print(i)
        if debug:
            print(f.toarray()[0])
            print(tt_flow)
            print(tt_p)
            print()
            print(potential(graph,f.toarray()[0])-potential(graph, (1. - s) * f.toarray()[0] + s * f_aon.toarray()))
            print(s)
    if debug:
        print(h)
    return h

In [84]:
eps=1e-10
nb_iter = 1000
c = [100, 100, 2, 100, 100]

h = solver(eps, nb_iter, demand, delta_csr, c)

f = h @ delta_csr
tt_flow = travel_time(f.toarray()[0], c)
tt_p = delta_csr @ tt_flow

if debug:
    print(h)
    print(tt_p[h.toarray()[0]!=0])
    print(tt_flow)
    print(delta_csr @ tt_flow)
    print(np.argsort(delta_csr @ tt_flow))
    print(h.toarray()[0])